In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import lightgbm as lgb

In [3]:
path=Path('/kaggle/data_science_bowl')
path

PosixPath('/kaggle/data_science_bowl')

### Read Data

In [4]:
def read_data():
    train_df = pd.read_csv(path/'train.csv')
    test_df = pd.read_csv(path/'test.csv')
    train_labels_df = pd.read_csv(path/'train_labels.csv')
    specs_df = pd.read_csv(path/'specs.csv')
    return train_df, test_df, train_labels_df, specs_df

In [5]:
train_df, test_df, train_labels_df, specs_df = read_data()

In [6]:
train_df.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [32]:
test_df.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10T16:54:12.115Z,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES


In [34]:
train_labels_df.head()

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


In [ ]:
specs_df.head()

## Feature Engineering

### Process train set

In [8]:
main_keys = ['installation_id', 'game_session']
merge_args = {'left_index':True, 'right_index':True}

In [9]:
merged_train_df = pd.merge(train_df, train_labels_df, on=main_keys)

#### Experiment with aggregations

In [11]:
merged_train_df

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,num_correct,num_incorrect,accuracy,accuracy_group
0,3bfd1a65,901acc108f55a5a1,2019-08-06T05:22:01.344Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,1,0,1.000000,3
1,db02c830,901acc108f55a5a1,2019-08-06T05:22:01.400Z,"{""event_count"":2,""game_time"":37,""event_code"":2...",0006a69f,2,2025,37,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,1,0,1.000000,3
2,a1e4395d,901acc108f55a5a1,2019-08-06T05:22:01.403Z,"{""description"":""Pull three mushrooms out of th...",0006a69f,3,3010,37,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,1,0,1.000000,3
3,a52b92d5,901acc108f55a5a1,2019-08-06T05:22:05.242Z,"{""description"":""Pull three mushrooms out of th...",0006a69f,4,3110,3901,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,1,0,1.000000,3
4,a1e4395d,901acc108f55a5a1,2019-08-06T05:22:05.244Z,"{""description"":""To pick a mushroom, pull it ou...",0006a69f,5,3010,3901,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,1,0,1.000000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865442,28520915,5448d652309a6324,2019-09-22T02:07:27.562Z,"{""misses"":1,""prompt"":""holds least"",""mode"":""sel...",ffeb0b1b,58,2030,67094,Cauldron Filler (Assessment),Assessment,MAGMAPEAK,1,2,0.333333,1
865443,91561152,5448d652309a6324,2019-09-22T02:07:27.562Z,"{""bucket"":1,""buckets_placed"":[3,1,2],""target_b...",ffeb0b1b,57,4025,67094,Cauldron Filler (Assessment),Assessment,MAGMAPEAK,1,2,0.333333,1
865444,d3268efa,5448d652309a6324,2019-09-22T02:07:27.566Z,"{""description"":""Awesome."",""identifier"":""Dot_Aw...",ffeb0b1b,59,3021,67094,Cauldron Filler (Assessment),Assessment,MAGMAPEAK,1,2,0.333333,1
865445,b5053438,5448d652309a6324,2019-09-22T02:07:28.311Z,"{""description"":""Awesome."",""identifier"":""Dot_Aw...",ffeb0b1b,60,3121,67847,Cauldron Filler (Assessment),Assessment,MAGMAPEAK,1,2,0.333333,1


In [17]:
merged_train_grouped_df = merged_train_df.groupby(['event_id', 'game_session', 'installation_id', 'title'])

In [29]:
merged_train_agg_df = merged_train_grouped_df.agg({'game_time': ['sum', 'mean'],
                                                   'event_code': lambda x:x.value_counts().index[0],
                                                   'event_count': ['sum', 'mean']})

In [31]:
merged_train_agg_df.reset_index()

event_id      game_session installation_id  \
                                                     
0       04df9b66  0067974a3f62e5b0        406ebb06   
1       04df9b66  006ec667711e7708        7d9d60e6   
2       04df9b66  008a884d5604084d        cf2a3a60   
3       04df9b66  010047c9b14b7fcf        5e9f2830   
4       04df9b66  0106e25f5c2bf40d        8cd1cf19   
...          ...               ...             ...   
241607  fbaf3456  ffde5ebf5d6df684        b0362800   
241608  fbaf3456  ffe06fe14beb0276        e34f8b5c   
241609  fbaf3456  ffe4149114747eb7        996c185b   
241610  fbaf3456  ffe43e55c3f6a209        78cacfd1   
241611  fbaf3456  fff641eaf483ae2b        bac91c32   

                               title game_time               event_code  \
                                           sum          mean   <lambda>   
0       Cauldron Filler (Assessment)      4153   4153.000000       3020   
1       Cauldron Filler (Assessment)     31756  31756.000000       3020   
2       Cauldron Filler (Assessment)     20611  20611.000000       3020   
3       Cauldron Filler (Assessment)     66215  22071.666667       3020   
4       Cauldron Filler (Assessment)     48662  24331.000000       3020   
...                              ...       ...           ...        ...   
241607  Mushroom Sorter (Assessment)    121691  24338.200000       4030   
241608  Mushroom Sorter (Assessment)     34656  11552.000000       4030   
241609  Mushroom Sorter (Assessment)    111164  18527.333333       4030   
241610  Mushroom Sorter (Assessment)    502126  45647.818182       4030   
241611  Mushroom Sorter (Assessment)     78953  26317.666667       4030   

       event_count             
               sum       mean  
0                6   6.000000  
1               31  31.000000  
2               26  26.000000  
3               87  29.000000  
4               55  27.500000  
...            ...        ...  
241607         130  26.000000  
241608          69  23.000000  
241609         164  27.333333  
241610         509  46.272727  
241611          81  27.000000  

[241612 rows x 9 columns]

In [30]:
def extract_time_features(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['month'] = df['timestamp'].dt.month
    df['hour'] = df['timestamp'].dt.hour
    df['year'] = df['timestamp'].dt.year
    df['dayofweek'] = df['timestamp'].dt.dayofweek
    df['weekofyear'] = df['timestamp'].dt.weekofyear
    df['dayofyear'] = df['timestamp'].dt.dayofyear
    df['quarter'] = df['timestamp'].dt.quarter
    return df

merged_train_df = extract_time_features(merged_train_df)
merged_train_df.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,...,num_incorrect,accuracy,accuracy_group,month,hour,year,dayofweek,weekofyear,dayofyear,quarter
0,3bfd1a65,901acc108f55a5a1,2019-08-06 05:22:01.344000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Mushroom Sorter (Assessment),Assessment,...,0,1.0,3,8,5,2019,1,32,218,3
1,db02c830,901acc108f55a5a1,2019-08-06 05:22:01.400000+00:00,"{""event_count"":2,""game_time"":37,""event_code"":2...",0006a69f,2,2025,37,Mushroom Sorter (Assessment),Assessment,...,0,1.0,3,8,5,2019,1,32,218,3
2,a1e4395d,901acc108f55a5a1,2019-08-06 05:22:01.403000+00:00,"{""description"":""Pull three mushrooms out of th...",0006a69f,3,3010,37,Mushroom Sorter (Assessment),Assessment,...,0,1.0,3,8,5,2019,1,32,218,3
3,a52b92d5,901acc108f55a5a1,2019-08-06 05:22:05.242000+00:00,"{""description"":""Pull three mushrooms out of th...",0006a69f,4,3110,3901,Mushroom Sorter (Assessment),Assessment,...,0,1.0,3,8,5,2019,1,32,218,3
4,a1e4395d,901acc108f55a5a1,2019-08-06 05:22:05.244000+00:00,"{""description"":""To pick a mushroom, pull it ou...",0006a69f,5,3010,3901,Mushroom Sorter (Assessment),Assessment,...,0,1.0,3,8,5,2019,1,32,218,3


In [ ]:
# Remove the timestamp column
merged_train_df = merged_train_df[[col for col in merged_train_df.columns if col not in ['timestamp', 'num_correct', 'num_incorrect', 'accuracy']]]
merged_train_df.head()

In [ ]:
merged_train_df.columns

In [ ]:
def convert_categorical_to_num(df, categorical_list):
    for cat_name in categorical_list:
        df[cat_name] = df[cat_name].astype('category')
    # Get categorical columns again
    cat_columns = df.select_dtypes(['category']).columns
    print('cat_columns', cat_columns)
    df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)

convert_categorical_to_num(merged_train_df, ['title', 'type', 'world', 'event_id'])

In [ ]:
pd.options.display.max_columns = None
merged_train_df

### Process test set

In [ ]:
comp_test_df = extract_time_features(test_df)
comp_test_df.head()

In [ ]:
comp_test_df = comp_test_df[[col for col in comp_test_df.columns if col not in ['timestamp']]]
comp_test_df.head()

In [ ]:
convert_categorical_to_num(comp_test_df, ['title', 'type', 'world', 'event_id'])
comp_test_df.head()

#### Checks

In [ ]:
comp_test_df.columns

In [ ]:
merged_train_df.columns

In [ ]:
merged_train_df.shape

In [ ]:
comp_test_df.shape

In [ ]:
comp_test_df['installation_id'].nunique()

## Training

In [ ]:
# quadratic weighted kappa
def qwk3(a1, a2, max_rat=3):
    '''
    a1 - ground truth
    a2 - predicted values
    '''
    assert(len(a1) == len(a2))
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)
    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))
    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)
    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)
    e = e / a1.shape[0]
    return 1 - o / e

In [ ]:
features = [c for c in merged_train_df.columns if c not in ['accuracy_group', 'installation_id', 'game_session', 'event_data']]
target = 'accuracy_group'

In [ ]:
features

In [ ]:
num_splits = 10
params = {
    'learning_rate': 0.007,
    'metric': 'multiclass',
    'objective': 'multiclass',
    'num_classes': 4,
    'feature_fraction': 0.75,
    "bagging_fraction": 0.8,
    "bagging_seed": 42,
    'max_depth': 11
}

early_stopping_rounds = 100
num_boost_round = 1000 # should be around 11000 to get better results. This is just for quick testing

In [ ]:
def train_model(comp_train_df):
    
    kf = KFold(n_splits=num_splits, shuffle=True)
    
    oof_pred = np.zeros((len(comp_train_df), 4))
    models = []
    best_qw3 = 0.0
    best_qw3_index = 0
    
    for fold, (tr_ind, val_ind) in enumerate(kf.split(comp_train_df)):
        print(f'Fold: {fold+1}')
        x_train, x_val = comp_train_df[features].iloc[tr_ind], comp_train_df[features].iloc[val_ind]
        y_train, y_val = comp_train_df[target][tr_ind], comp_train_df[target][val_ind]
        train_set = lgb.Dataset(x_train, y_train)
        val_set = lgb.Dataset(x_val, y_val)

        model = lgb.train(params, train_set, num_boost_round = num_boost_round, early_stopping_rounds = early_stopping_rounds, 
                          valid_sets=[train_set, val_set], verbose_eval = early_stopping_rounds)
        oof_pred[val_ind] = model.predict(x_val)
        models.append(model)
        
        val_crt_fold = qwk3(y_val, oof_pred[val_ind].argmax(axis = 1))
        print(f'Fold: {fold+1} quadratic weighted kappa score: {np.round(val_crt_fold,4)}')
        if best_qw3 < val_crt_fold:
            best_qw3 = val_crt_fold
            best_qw3_index = fold
        
    res = qwk3(comp_train_df['accuracy_group'], oof_pred.argmax(axis = 1))
    print(f'Quadratic weighted score: {np.round(res,4)}')
        
    return models, best_qw3_index

In [ ]:
%%time
all_models, best_qw3_index = train_model(merged_train_df)
print(f'best model index: {best_qw3_index}')

## Inference

In [ ]:
grouped = comp_test_df.groupby('installation_id')
comp_test_tail_df = pd.concat([grouped.tail(1)]).sort_values('installation_id').reset_index(drop=True)

In [ ]:
comp_test_tail_df

In [ ]:
def run_predictions(models, df):
    y_pred = np.zeros((len(df), 4))
    for i, model in enumerate(models):
        y_pred += model.predict(df[features])
        print(f'Ran {i}th model ')
    return y_pred / num_splits

In [ ]:
y_pred = run_predictions(all_models, comp_test_tail_df)

In [ ]:
# y_pred = all_models[best_qw3_index].predict(comp_test_df[features])

In [ ]:
assert comp_test_tail_df.shape[0] == y_pred.shape[0]

In [ ]:
np.unique(y_pred.argmax(-1), return_counts=True)

In [ ]:
comp_test_tail_df['accuracy_group'] = y_pred.argmax(-1)

In [ ]:
comp_test_tail_df.shape, comp_test_tail_df.columns

### Create Submission

In [ ]:
def prepare_submission(submission_df, y_pred):
    sample_submission_df = pd.read_csv(path/'sample_submission.csv')
    sample_submission_df.drop('accuracy_group', inplace = True, axis = 1)
    sample_submission_df = sample_submission_df.merge(submission_df, on = 'installation_id')
    sample_submission_df = sample_submission_df[['installation_id', 'accuracy_group']]
    sample_submission_df.to_csv('submission.csv', index = False)

In [ ]:
prepare_submission(comp_test_tail_df, y_pred)

In [ ]:
!head submission.csv

In [ ]:
!cat submission.csv | wc -l